In [2]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier

In [2]:
data = pd.read_csv("Data.csv")
data

,Medicine Name,Composition,Uses,Side_effects,Image URL,Manufacturer,Excellent Review %,Average Review %,Poor Review %
0,Avastin 400mg Injection,Bevacizumab (400mg),Cancer of colon and rectum Non-small cell lun...,Rectal bleeding Taste change Headache Noseblee...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Roche Products India Pvt Ltd,22,56,22
1,Augmentin 625 Duo Tablet,Amoxycillin (500mg) + Clavulanic Acid (125mg),Treatment of Bacterial infections,Vomiting Nausea Diarrhea Mucocutaneous candidi...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Glaxo SmithKline Pharmaceuticals Ltd,47,35,18
2,Azithral 500 Tablet,Azithromycin (500mg),Treatment of Bacterial infections,Nausea Abdominal pain Diarrhea,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Alembic Pharmaceuticals Ltd,39,40,21
3,Ascoril LS Syrup,Ambroxol (30mg/5ml) + Levosalbutamol (1mg/5ml)...,Treatment of Cough with mucus,Nausea Vomiting Diarrhea Upset stomach Stomach...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Glenmark Pharmaceuticals Ltd,24,41,35
4,Aciloc 150 Tablet,Ranitidine (150mg),Treatment of Gastroesophageal reflux disease (...,Headache Diarrhea Gastrointestinal disturbance,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Cadila Pharmaceuticals Ltd,34,37,29
...,...,...,...,...,...,...,...,...,...
11820,Zilarta-CT 40/6.25 Tablet,Azilsartan medoxomil (40mg) + Chlorthalidone (...,Hypertension (high blood pressure),Dizziness Tiredness Decreased potassium level ...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Micro Labs Ltd,67,33,0
11821,Zipcoz Tablet,Chromium Picolinate (100mcg) + D-Chiro Inosito...,Polycystic ovarian syndrome (PCOS),Rash Nausea Paresthesia tingling or pricking s...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Lifeline Remedies India Pvt Ltd,0,80,20
11822,Zestasil 100 Tablet,Sildenafil (100mg),Treatment of Erectile dysfunction,Flushing sense of warmth in the face ears neck...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",TBG pharma ltd,25,75,0
11823,Zedruff Shampoo,Ketoconazole (2% w/v) + Zinc pyrithione (1% w/v),Treatment of Dandruff,Application site reactions burning irritation ...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Minova Life Sciences Pvt Ltd,0,0,100


In [3]:
data.isnull().sum()

Medicine Name         0
Composition           0
Uses                  0
Side_effects          0
Image URL             0
Manufacturer          0
Excellent Review %    0
Average Review %      0
Poor Review %         0
dtype: int64

In [4]:
data.shape

(11825, 9)

In [5]:
data.describe

<bound method NDFrame.describe of                    Medicine Name  \
0        Avastin 400mg Injection   
1       Augmentin 625 Duo Tablet   
2            Azithral 500 Tablet   
3               Ascoril LS Syrup   
4              Aciloc 150 Tablet   
...                          ...   
11820  Zilarta-CT 40/6.25 Tablet   
11821              Zipcoz Tablet   
11822        Zestasil 100 Tablet   
11823            Zedruff Shampoo   
11824            Zedruff Shampoo   

                                             Composition  \
0                                    Bevacizumab (400mg)   
1        Amoxycillin  (500mg) +  Clavulanic Acid (125mg)   
2                                   Azithromycin (500mg)   
3      Ambroxol (30mg/5ml) + Levosalbutamol (1mg/5ml)...   
4                                     Ranitidine (150mg)   
...                                                  ...   
11820  Azilsartan medoxomil (40mg) + Chlorthalidone (...   
11821  Chromium Picolinate (100mcg) + D-Chiro Inosito

In [ ]:
import re
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Step 1: Parse composition to extract ingredients and dosages
def parse_composition(composition):
    matches = re.findall(r'([\w\s]+)\(([\d./mg]+)\)', str(composition))
    return {ingredient.strip(): dosage for ingredient, dosage in matches}

data['Parsed_Composition'] = data['Composition'].apply(parse_composition)

# Step 2: Standardize dosages
def standardize_dosage(parsed_composition):
    standardized = {}
    for ingredient, dosage in parsed_composition.items():
        try:
            if '/' in dosage:
                # Standardize mixed dosages (e.g., 30mg/5ml)
                amounts = re.findall(r'[\d.]+', dosage)
                if len(amounts) == 2:
                    amount, per_unit = map(float, amounts)
                    unit = re.search(r'[a-zA-Z/]+', dosage).group()
                    standardized[ingredient] = f"{amount / per_unit:.2f}{unit.split('/')[0]}/{unit.split('/')[1]}"
                else:
                    standardized[ingredient] = dosage  # Leave as-is if unclear
            else:
                standardized[ingredient] = dosage
        except:
            standardized[ingredient] = dosage  # Fallback to original
    return standardized

data['Standardized_Composition'] = data['Parsed_Composition'].apply(standardize_dosage)

# Step 3: Prepare side effects (handling multi-word side effects)
def split_side_effects(side_effects):
    return re.split(r',\s*|\s{2,}', str(side_effects).strip())

data['Side_effects_List'] = data['Side_effects'].apply(split_side_effects)

# Step 4: Binarize side effects for multi-label classification
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(data['Side_effects_List'])

# Convert standardized composition to a single string for vectorization
data['Composition_String'] = data['Standardized_Composition'].apply(lambda x: ' '.join([f"{k}:{v}" for k, v in x.items()]))

# Vectorize the composition string
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['Composition_String'])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Train a Random Forest model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Step 6: Evaluate the model
y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred, target_names=mlb.classes_)

# Output the classification report
report

In [ ]:
new_composition = {'Ambroxol': '30mg', 'Ranitidine' : '250mg'}
new_composition_string = ' '.join([f"{k}:{v}" for k, v in new_composition.items()])
new_composition_vectorized = vectorizer.transform([new_composition_string])
new_prediction = model.predict(new_composition_vectorized)

# Decode the predicted side effects
predicted_side_effects = mlb.inverse_transform(new_prediction)
print("Predicted Side Effects:", predicted_side_effects)

In [ ]:
model.score(X_test, y_test)

In [ ]:
data

In [ ]:
data.to_csv("FinalData.csv")

In [ ]:
data['Side_effects_List']

In [2]:
domain_to_side_effects1 = {
    'Neurological_effects' : [
    "Memory loss",
    "Confusion",
    "Dementia",
    "Delirium",
    "Paralysis",
    "Muscle weakness",
    "Tremor",
    "Seizure",
    "Headache",
    "Dizziness",
    "Numbness or tingling",
    "Loss of coordination",
    "Speech difficulties",
    "Vision disturbance",
    "Hearing loss",
    "Balance problem",
    "Chronic pain",
    "Fatigue",
    "Sleep disturbance",
    "Mood change",
    "Hallucination",
    "Hyperactivity",
    "Sensory overload",
    "Autonomic dysfunction",
    "Cognitive decline",
    "ataxia",
    "aphasia",
    "neuropathy",
    "anxiety",
    "depression",
    "ADHD symptom",
    "irregular heartbeat",
    "sweating issue",
    "difficulty concentrating"
],
    'Gastrointestinal_effects' : [
    "Nausea",
    "Vomiting",
    "Diarrhea",
    "Constipation",
    "Abdominal pain",
    "Bloating",
    "Indigestion",
    "Heartburn",
    "Loss of appetite",
    "Gastrointestinal bleeding",
    "Irritable bowel syndrome",
    "Acidic stomach",
    "Flatulence",
    "Swallowing difficulties",
    "Peptic ulcers",
    "Gallstones",
    "Jaundice",
    "Crohn’s disease",
    "Ulcerative colitis",
    "Malabsorption",
    "Hiccups",
    "Dehydration",
    "Hemorrhoids",
    "Intestinal obstruction",
    "Rectal bleeding",
    "dyspepsia",
    "acid reflux",
    "IBS",
    "excessive gas",
    "dysphagia", 
],
    'Cardiovascular_effects' : [
    "Chest pain",
    "Shortness of breath",
    "Palpitations",
    "Irregular heartbeat",
    "High blood pressure",
    "Low blood pressure",
    "Heart attack",
    "Heart failure",
    "Swelling in the legs or feet",
    "Dizziness or fainting",
    "Fatigue",
    "Poor circulation",
    "Blood clots",
    "Stroke",
    "Atherosclerosis",
    "Arterial narrowing",
    "Rapid heart rate",
    "Slow heart rate",
    "Peripheral artery disease",
    "Cardiac arrest",
    "Endocarditis",
    "Pericarditis",
    "Pulmonary embolism",
    "Varicose veins",
    "Aneurysm",
    "angina",
    "arrhythmia",
    "hypertension",
    "hypotension",
    "myocardial infarction",
    "edema",
    "syncope",
    "cold extremities",
    "thrombosis",
    "tachycardia",
    "bradycardia"
],
    'Dermatological_effects' :[
    "Rashes",
    "Itching",
    "Dry skin",
    "Redness",
    "Swelling",
    "Acne",
    "Hives",
    "Blisters",
    "Peeling skin",
    "Skin discoloration",
    "Hair loss",
    "Eczema",
    "Psoriasis",
    "Skin infections",
    "Cracked skin",
    "Bruising",
    "Warts",
    "Scarring",
    "Excessive sweating",
    "Skin sensitivity",
    "Ulcers",
    "Cold sores",
    "Sunburn",
    "Fungal infections",
    "Allergic dermatitis",
    "pruritus",
    "erythema",
    "edema",
    "urticaria",
    "hyperpigmentation",
    "alopecia",
    "hyperhidrosis"
],
    'Psychological_effects' : [
    "Anxiety",
    "Depression",
    "Stress",
    "Mood swings",
    "Irritability",
    "Panic attacks",
    "Obsessive-compulsive behaviors",
    "Phobias",
    "Post-traumatic stress disorder",
    "PTSD",
    "Bipolar disorder",
    "Schizophrenia",
    "Paranoia",
    "Hallucinations",
    "Delusions",
    "Social withdrawal",
    "Difficulty concentrating",
    "Memory problems",
    "Low self-esteem",
    "Suicidal thoughts",
    "Sleep disturbances",
    "insomnia",
    "hypersomnia",
    "Eating disorders",
    "anorexia",
    "bulimia",
    "Substance abuse",
    "Aggression",
    "Apathy",
    "Emotional numbness"
],
    'Respiratory_effects' : [
    "Difficulty breathing",
    "dyspnea",
    "Coughing",
    "Wheezing",
    "Asthma",
    "Chronic obstructive pulmonary disease",
    "COPD",
    "Respiratory infections",
    "Pneumonia",
    "Chest tightness",
    "Sleep apnea",
    "Pulmonary fibrosis"
],
    'Endocrine_effects' : [
    "Hormonal imbalances",
    "Thyroid dysfunction",
    "hyperthyroidism",
    "hypothyroidism",
    "Diabetes",
    "Adrenal fatigue",
    "Weight changes",
    "Growth disorders",
    "Polycystic ovary syndrome",
    "PCOS",
    "Metabolic syndrome",
    "Cushing's syndrome",
    "Hyperparathyroidism"
],
    'Musculoskeletal_effects' : [
    "Muscle cramps",
    "Joint pain",
    "arthritis",
    "Bone fractures",
    "Osteoporosis",
    "Muscle weakness",
    "Back pain",
    "Tendonitis",
    "Skeletal deformities",
    "Fibromyalgia",
    "Spinal disc herniation"
],
    'Renal_and_Urinary_effects' : [
    "Frequent urination",
    "Painful urination",
    "dysuria",
    "Kidney stones",
    "Urinary retention",
    "Bladder infections",
    "Proteinuria",
    "protein in urine",
    "Hematuria",
    "blood in urine",
    "Chronic kidney disease",
    "CKD",
    "Urinary incontinence",
    "Nephritis"
],
    'Immune_system_effects' : [
    "Allergic reactions",
    "Autoimmune diseases",
    "Infections",
    "Chronic inflammation",
    "Immunodeficiency",
    "Fever",
    "Lymph node swelling",
    "Delayed wound healing",
    "Sepsis",
    "Recurrent infections"
]
}

In [3]:
domain_to_side_effects

NameError: name 'domain_to_side_effects' is not defined

In [4]:
domain_to_side_effects2 = {
    "Neurological_effects": [
        "Agitation",
        "Akathisia",
        "Affect",
        "Anger",
        "Anxiety",
        "Attention",
        "Azoospermia",
        "Balance",
        "Behavior",
        "Behavioral",
        "Brain",
        "Cerebral",
        "Confusion",
        "Confusional",
        "Convulsion",
        "Coordination",
        "Dizziness",
        "Deprivation",
        "Disorientation",
        "Delayed",
        "Disturbance",
        "Dystonia",
        "Dreams",
        "Drowsiness",
        "Extrapyramidal",
        "Failure",
        "Fainting",
        "Fear",
        "Gait",
        "Hallucination",
        "Hemiparesis",
        "Hearing",
        "Hypoesthesia",
        "Hypersomnia",
        "Memory",
        "Migraine",
        "Mood",
        "Nervousness",
        "Neuropathy",
        "Neuropsychiatric",
        "Nightmares",
        "Nystagmus",
        "Tingling",
        "Trembling",
        "Tremors",
        "Twitching"
    ],
    "Gastrointestinal_effects": [
        "Abdominal",
        "Ache",
        "Anorectal",
        "Appetite",
        "Ascites",
        "Bowel",
        "Bowels",
        "Bloating",
        "Burping",
        "Bladder",
        "Chalky",
        "Cholesterol",
        "Cold",
        "Colon",
        "Color",
        "Cavity",
        "Constipation",
        "Diarrhea",
        "Dyspepsia",
        "Dehydration",
        "Depletion",
        "Discomfort",
        "Epigastric",
        "Excessive",
        "Evacuation",
        "Flatulence",
        "Flatus",
        "Fecal",
        "Fluid",
        "Fullness",
        "Gastric",
        "Gastroenteritis",
        "Gastrointestinal",
        "Glucose",
        "Indigestion",
        "Micturition",
        "Nausea",
        "Thirst",
        "Thirsty"
    ],
    "Cardiovascular_effects": [
        "Angina",
        "Angioedema",
        "Arrhythmia",
        "Arrhythmias",
        "Atrial",
        "Atrioventricular",
        "Attack",
        "Block",
        "Blood",
        "Breathlessness",
        "Cardiac",
        "Cavity",
        "Clot",
        "Coagulation",
        "Chronic",
        "Cholesterol",
        "Cerebral",
        "Elevated",
        "Extrasystoles",
        "Fibrillation",
        "Febrile",
        "Flushes",
        "Flushing",
        "Failure",
        "Frequent",
        "Frequency",
        "Heart",
        "Heartbeats",
        "Heartburn",
        "Hemoglobin",
        "Hemorrhage",
        "Hypertension",
        "Hypotension",
        "Tachycardia",
        "Tachysystole"
    ],
    "Dermatological_effects": [
        "Acne",
        "Alteration",
        "Altered",
        "Blanching",
        "Blisters",
        "Bluish",
        "Bruising",
        "Burn",
        "Burning",
        "Bumps",
        "Bruise",
        "Brightness",
        "Black",
        "Blue",
        "Cutaneous",
        "Cysts",
        "Coloration",
        "Colored",
        "Conjunctival",
        "Conjunctivitis",
        "Dermatitis",
        "Eczema",
        "Ecchymosis",
        "Eruption",
        "Exfoliation",
        "Erythema",
        "Erythematous",
        "Fissuring",
        "Flakes",
        "Follicle",
        "Follicular",
        "Gingival",
        "Goosebumps",
        "Grey",
        "Hives",
        "Hot",
        "Itching",
        "Itchy",
        "Maculopapular",
        "Rash",
        "Red",
        "Redness",
        "Scaling",
        "Scalp",
        "Scleral",
        "Skin",
        "Teeth",
        "Telangiectasia",
        "Ulcer",
        "Ulceration",
        "Ulcerative"
    ],
    "Psychological_effects": [
        "Aggressive",
        "Anger",
        "Anorgasmia",
        "Anxiety",
        "Behavior",
        "Behavioral",
        "Chronic",
        "Conditions",
        "Depression",
        "Deficiency",
        "Deprivation",
        "Dizziness",
        "Dreams",
        "Decreased",
        "Disturbances",
        "Euphoria",
        "Excitement",
        "Fear",
        "Feeling",
        "Fainting",
        "Happiness",
        "Hallucination",
        "Hyperactivity",
        "Mental",
        "Mood",
        "Thoughts",
        "Restlessness"
    ],
    "Respiratory_effects": [
        "Airway",
        "Airways",
        "Apnea",
        "Asthma",
        "Breath",
        "Breathing",
        "Bronchial",
        "Bronchitis",
        "Bronchospasm",
        "Cough",
        "Coughing",
        "Dyslipidemia",
        "Flu",
        "Hiccup",
        "Hoarseness",
        "Ringing",
        "Runny",
        "Sneezing",
        "Throat",
        "Wheezing"
    ],
    "Endocrine_effects": [
        "Adrenal",
        "Cushing",
        "Diabetes",
        "Endometrial",
        "Elevated",
        "Enzyme",
        "Enzymes",
        "Goiter",
        "Glutamic",
        "Glutamyltransferase",
        "Hyperplasia",
        "Hypoglycaemia",
        "Hypoglycemia",
        "Hyperstimulation",
        "Hypothyroidism",
        "Hypotension",
        "Thyroid",
        "Toxicity"
    ],
    "Musculoskeletal_effects": [
        "Ankle",
        "Atrophy",
        "Back",
        "Bone",
        "Cramps",
        "Cramp",
        "Contractions",
        "Damage",
        "Dystonia",
        "Engorgement",
        "Enlargement",
        "Enlarged",
        "Extremities",
        "Extremity",
        "Fatigue",
        "Fat",
        "Feet",
        "Fingers",
        "Growth",
        "Hypertrophy",
        "Leg",
        "Muscle",
        "Musculoskeletal",
        "Tenderness",
        "Tissue",
        "Tone"
    ],
    "Renal_and_Urinary_effects": [
        "Absence",
        "Azoospermia",
        "Bladder",
        "Creatine",
        "Creatinine",
        "Calcium",
        "Dehydration",
        "Dehydrogenase",
        "Electrolyte",
        "Glycosuria",
        "Incontinence",
        "Injury",
        "Kidney",
        "Imbalance",
        "Infections",
        "Inflammation",
        "Ischemic",
        "Microhematuria",
        "Nephrocalcinosis",
        "Nephrotic",
        "Nocturia",
        "Retention",
        "Testicular",
        "Tests",
        "Test",
        "Urea",
        "Urge",
        "Urinary",
        "Urine",
        "Urinating",
        "Urination",
        "Urticaria",
        "Uterine"
    ]
}

In [11]:
# Combine the dictionaries by merging values for matching keys without duplicates
combined_dictionary = {}

for key in set(domain_to_side_effects1.keys()).union(domain_to_side_effects2.keys()):
    combined_dictionary[key] = list(
        set(domain_to_side_effects1.get(key, []) + domain_to_side_effects2.get(key, []))
    )

combined_dictionary


{'Musculoskeletal_effects': ['Feet',
  'Cramps',
  'Bone fractures',
  'arthritis',
  'Dystonia',
  'Ankle',
  'Engorgement',
  'Fatigue',
  'Tenderness',
  'Bone',
  'Extremity',
  'Back pain',
  'Atrophy',
  'Muscle',
  'Fibromyalgia',
  'Fat',
  'Extremities',
  'Spinal disc herniation',
  'Enlargement',
  'Leg',
  'Cramp',
  'Fingers',
  'Back',
  'Musculoskeletal',
  'Osteoporosis',
  'Tissue',
  'Muscle weakness',
  'Joint pain',
  'Growth',
  'Tone',
  'Hypertrophy',
  'Tendonitis',
  'Muscle cramps',
  'Enlarged',
  'Skeletal deformities',
  'Contractions',
  'Damage'],
 'Immune_system_effects': ['Chronic inflammation',
  'Fever',
  'Sepsis',
  'Immunodeficiency',
  'Allergic reactions',
  'Autoimmune diseases',
  'Delayed wound healing',
  'Recurrent infections',
  'Lymph node swelling',
  'Infections'],
 'Gastrointestinal_effects': ['Depletion',
  'Hiccups',
  'Color',
  'excessive gas',
  'Bowels',
  'Dehydration',
  'Abdominal pain',
  'Ulcerative colitis',
  'Evacuation',


In [5]:
combined_dictionary = {'Musculoskeletal_effects': ['Feet',
  'Cramps',
  'Bone fractures',
  'arthritis',
  'Dystonia',
  'Ankle',
  'Engorgement',
  'Fatigue',
  'Tenderness',
  'Bone',
  'Extremity',
  'Back pain',
  'Atrophy',
  'Muscle',
  'Fibromyalgia',
  'Fat',
  'Extremities',
  'Spinal disc herniation',
  'Enlargement',
  'Leg',
  'Cramp',
  'Fingers',
  'Back',
  'Musculoskeletal',
  'Osteoporosis',
  'Tissue',
  'Muscle weakness',
  'Joint pain',
  'Growth',
  'Tone',
  'Hypertrophy',
  'Tendonitis',
  'Muscle cramps',
  'Enlarged',
  'Skeletal deformities',
  'Contractions',
  'Damage'],
 'Immune_system_effects': ['Chronic inflammation',
  'Fever',
  'Sepsis',
  'Immunodeficiency',
  'Allergic reactions',
  'Autoimmune diseases',
  'Delayed wound healing',
  'Recurrent infections',
  'Lymph node swelling',
  'Infections'],
 'Gastrointestinal_effects': ['Depletion',
  'Hiccups',
  'Color',
  'excessive gas',
  'Bowels',
  'Dehydration',
  'Abdominal pain',
  'Ulcerative colitis',
  'Evacuation',
  'Colon',
  'Diarrhea',
  'Fecal',
  'Gastric',
  'Acidic stomach',
  'Malabsorption',
  'acid reflux',
  'Abdominal',
  'Heartburn',
  'Chalky',
  'Cold',
  'Fullness',
  'Crohn’s disease',
  'Gallstones',
  'Peptic ulcers',
  'Cholesterol',
  'Cavity',
  'Hemorrhoids',
  'Epigastric',
  'Flatulence',
  'Fluid',
  'Appetite',
  'IBS',
  'Excessive',
  'Thirst',
  'dyspepsia',
  'Bladder',
  'Loss of appetite',
  'Burping',
  'dysphagia',
  'Ascites',
  'Flatus',
  'Bowel',
  'Bloating',
  'Gastrointestinal bleeding',
  'Intestinal obstruction',
  'Swallowing difficulties',
  'Micturition',
  'Ache',
  'Discomfort',
  'Vomiting',
  'Constipation',
  'Glucose',
  'Indigestion',
  'Irritable bowel syndrome',
  'Dyspepsia',
  'Anorectal',
  'Gastroenteritis',
  'Thirsty',
  'Gastrointestinal',
  'Jaundice',
  'Rectal bleeding',
  'Nausea'],
 'Neurological_effects': ['anxiety',
  'Chronic pain',
  'Seizure',
  'Brain',
  'Behavioral',
  'Azoospermia',
  'Hypersomnia',
  'Nystagmus',
  'Paralysis',
  'Vision disturbance',
  'Nervousness',
  'Hypoesthesia',
  'Hyperactivity',
  'Hallucination',
  'Neuropathy',
  'Loss of coordination',
  'Nightmares',
  'Trembling',
  'irregular heartbeat',
  'Tremors',
  'Balance problem',
  'Agitation',
  'Speech difficulties',
  'Extrapyramidal',
  'Dystonia',
  'aphasia',
  'Gait',
  'Mood',
  'Sleep disturbance',
  'Hemiparesis',
  'Disturbance',
  'Delirium',
  'Hearing',
  'Dreams',
  'Memory',
  'Disorientation',
  'Migraine',
  'Anxiety',
  'Sensory overload',
  'Dizziness',
  'sweating issue',
  'Confusion',
  'difficulty concentrating',
  'Neuropsychiatric',
  'Attention',
  'Deprivation',
  'Hearing loss',
  'Autonomic dysfunction',
  'Muscle weakness',
  'Balance',
  'ataxia',
  'Akathisia',
  'Cognitive decline',
  'Twitching',
  'Fainting',
  'Fatigue',
  'Mood change',
  'Behavior',
  'Tremor',
  'Dementia',
  'Memory loss',
  'Tingling',
  'Cerebral',
  'neuropathy',
  'Confusional',
  'Drowsiness',
  'depression',
  'Coordination',
  'Delayed',
  'Affect',
  'Failure',
  'Headache',
  'Fear',
  'Numbness or tingling',
  'ADHD symptom',
  'Convulsion',
  'Anger'],
 'Respiratory_effects': ['Coughing',
  'Airway',
  'Chronic obstructive pulmonary disease',
  'Sleep apnea',
  'Sneezing',
  'Bronchospasm',
  'Flu',
  'Hoarseness',
  'Bronchitis',
  'Bronchial',
  'Breath',
  'Runny',
  'Cough',
  'Dyslipidemia',
  'Chest tightness',
  'Ringing',
  'dyspnea',
  'Apnea',
  'Throat',
  'Respiratory infections',
  'COPD',
  'Airways',
  'Hiccup',
  'Difficulty breathing',
  'Pulmonary fibrosis',
  'Wheezing',
  'Pneumonia',
  'Breathing',
  'Asthma'],
 'Psychological_effects': ['Anxiety',
  'Memory problems',
  'Fainting',
  'Behavioral',
  'PTSD',
  'insomnia',
  'Dizziness',
  'Restlessness',
  'Aggressive',
  'Behavior',
  'Excitement',
  'Mood swings',
  'Anorgasmia',
  'Schizophrenia',
  'Mood',
  'anorexia',
  'Paranoia',
  'Apathy',
  'Deficiency',
  'Disturbances',
  'Post-traumatic stress disorder',
  'Social withdrawal',
  'Chronic',
  'Emotional numbness',
  'Decreased',
  'Deprivation',
  'Hyperactivity',
  'Hallucination',
  'Mental',
  'Euphoria',
  'Depression',
  'Bipolar disorder',
  'Delusions',
  'bulimia',
  'Irritability',
  'Low self-esteem',
  'hypersomnia',
  'Dreams',
  'Hallucinations',
  'Obsessive-compulsive behaviors',
  'Substance abuse',
  'Happiness',
  'Thoughts',
  'Conditions',
  'Aggression',
  'Stress',
  'Difficulty concentrating',
  'Panic attacks',
  'Sleep disturbances',
  'Suicidal thoughts',
  'Fear',
  'Feeling',
  'Phobias',
  'Anger',
  'Eating disorders'],
 'Endocrine_effects': ['Enzymes',
  'Growth disorders',
  'Cushing',
  'PCOS',
  'Toxicity',
  'hyperthyroidism',
  'Thyroid dysfunction',
  'Hypoglycaemia',
  'Diabetes',
  'Hyperplasia',
  'Glutamic',
  'Adrenal',
  'Weight changes',
  'Adrenal fatigue',
  'Hormonal imbalances',
  'Enzyme',
  'Hypothyroidism',
  'Polycystic ovary syndrome',
  'Goiter',
  'Hypotension',
  'Metabolic syndrome',
  'Hyperparathyroidism',
  'Hypoglycemia',
  'Hyperstimulation',
  'Glutamyltransferase',
  'Thyroid',
  'hypothyroidism',
  'Endometrial',
  "Cushing's syndrome",
  'Elevated'],
 'Dermatological_effects': ['Cracked skin',
  'Cysts',
  'Cold sores',
  'Ulcers',
  'Bruising',
  'Ecchymosis',
  'Colored',
  'Redness',
  'Altered',
  'Scarring',
  'Blanching',
  'Excessive sweating',
  'Bruise',
  'Conjunctival',
  'Cutaneous',
  'Grey',
  'Sunburn',
  'Rashes',
  'Blisters',
  'Acne',
  'Blue',
  'Teeth',
  'Bluish',
  'Fungal infections',
  'urticaria',
  'Follicle',
  'alopecia',
  'Bumps',
  'erythema',
  'Maculopapular',
  'Red',
  'edema',
  'Itchy',
  'hyperpigmentation',
  'Black',
  'Exfoliation',
  'Swelling',
  'pruritus',
  'Skin infections',
  'Burn',
  'Rash',
  'Scleral',
  'Coloration',
  'Scaling',
  'Conjunctivitis',
  'hyperhidrosis',
  'Goosebumps',
  'Eruption',
  'Erythema',
  'Hot',
  'Fissuring',
  'Telangiectasia',
  'Brightness',
  'Warts',
  'Scalp',
  'Ulcer',
  'Hives',
  'Psoriasis',
  'Allergic dermatitis',
  'Erythematous',
  'Skin discoloration',
  'Eczema',
  'Dermatitis',
  'Gingival',
  'Follicular',
  'Ulcerative',
  'Itching',
  'Dry skin',
  'Hair loss',
  'Ulceration',
  'Flakes',
  'Skin',
  'Burning',
  'Alteration',
  'Skin sensitivity',
  'Peeling skin'],
 'Cardiovascular_effects': ['Arterial narrowing',
  'Slow heart rate',
  'myocardial infarction',
  'Heartbeats',
  'Heart failure',
  'Atrial',
  'Hypertension',
  'Cardiac',
  'Flushing',
  'Fatigue',
  'Block',
  'Frequency',
  'Febrile',
  'Blood clots',
  'Pericarditis',
  'Tachycardia',
  'Heart',
  'Heartburn',
  'Chest pain',
  'Cardiac arrest',
  'Varicose veins',
  'syncope',
  'thrombosis',
  'Irregular heartbeat',
  'Arrhythmias',
  'Palpitations',
  'Cavity',
  'Cholesterol',
  'Dizziness or fainting',
  'Swelling in the legs or feet',
  'Aneurysm',
  'Frequent',
  'edema',
  'Attack',
  'Low blood pressure',
  'angina',
  'Stroke',
  'High blood pressure',
  'hypotension',
  'Cerebral',
  'arrhythmia',
  'Chronic',
  'Fibrillation',
  'Poor circulation',
  'Atrioventricular',
  'Angina',
  'Clot',
  'Coagulation',
  'Endocarditis',
  'Hypotension',
  'cold extremities',
  'Failure',
  'Hemorrhage',
  'Shortness of breath',
  'Heart attack',
  'Atherosclerosis',
  'Arrhythmia',
  'Peripheral artery disease',
  'Angioedema',
  'Flushes',
  'Hemoglobin',
  'hypertension',
  'bradycardia',
  'Pulmonary embolism',
  'Blood',
  'Tachysystole',
  'tachycardia',
  'Extrasystoles',
  'Elevated',
  'Breathlessness',
  'Rapid heart rate'],
 'Renal_and_Urinary_effects': ['Kidney stones',
  'Test',
  'Creatine',
  'Dehydration',
  'Painful urination',
  'Azoospermia',
  'Glycosuria',
  'Urinating',
  'Hematuria',
  'Proteinuria',
  'Urinary incontinence',
  'Kidney',
  'Nephrotic',
  'Electrolyte',
  'Microhematuria',
  'Imbalance',
  'Chronic kidney disease',
  'Testicular',
  'Retention',
  'blood in urine',
  'Bladder',
  'Urge',
  'Urine',
  'Urticaria',
  'Uterine',
  'protein in urine',
  'Calcium',
  'Dehydrogenase',
  'Nocturia',
  'Absence',
  'Infections',
  'Tests',
  'Nephritis',
  'Urination',
  'Nephrocalcinosis',
  'Urinary retention',
  'Bladder infections',
  'Incontinence',
  'Inflammation',
  'CKD',
  'Ischemic',
  'dysuria',
  'Creatinine',
  'Urinary',
  'Urea',
  'Frequent urination',
  'Injury']}

In [15]:
import pandas as pd

# Read the data
dt = pd.read_csv('FinalData.csv')

# Add a column for each domain and initialize with 0
for domain in combined_dictionary.keys():
    dt[domain] = 0

# Iterate through each row and check for side effects
for index, row in dt.iterrows():
    # Normalize the side effect list to lowercase and take only the first 5 characters of each side effect
    side_effect_list = [''.join(row['Side_effects_List']).lower() for _ in row['Side_effects_List'].split(' ')]
    
    # Check each domain
    for domain, effects in combined_dictionary.items():
        # Check if any of the first 5 characters of the side effect matches the domain's side effects
        if any(effect.lower() in [eff.lower() for eff in effects] for effect in side_effect_list):
            dt.at[index, domain] = 1


In [16]:
dt

,Unnamed: 0,Medicine Name,Composition,Uses,Side_effects,Image URL,Manufacturer,Excellent Review %,Average Review %,Poor Review %,...,Musculoskeletal_effects,Immune_system_effects,Gastrointestinal_effects,Neurological_effects,Respiratory_effects,Psychological_effects,Endocrine_effects,Dermatological_effects,Cardiovascular_effects,Renal_and_Urinary_effects
0,0,Avastin 400mg Injection,Bevacizumab (400mg),Cancer of colon and rectum Non-small cell lun...,Rectal bleeding Taste change Headache Noseblee...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Roche Products India Pvt Ltd,22,56,22,...,0,0,0,0,0,0,0,0,0,0
1,1,Augmentin 625 Duo Tablet,Amoxycillin (500mg) + Clavulanic Acid (125mg),Treatment of Bacterial infections,Vomiting Nausea Diarrhea Mucocutaneous candidi...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Glaxo SmithKline Pharmaceuticals Ltd,47,35,18,...,0,0,0,0,0,0,0,0,0,0
2,2,Azithral 500 Tablet,Azithromycin (500mg),Treatment of Bacterial infections,Nausea Abdominal pain Diarrhea,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Alembic Pharmaceuticals Ltd,39,40,21,...,0,0,0,0,0,0,0,0,0,0
3,3,Ascoril LS Syrup,Ambroxol (30mg/5ml) + Levosalbutamol (1mg/5ml)...,Treatment of Cough with mucus,Nausea Vomiting Diarrhea Upset stomach Stomach...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Glenmark Pharmaceuticals Ltd,24,41,35,...,0,0,0,0,0,0,0,0,0,0
4,4,Aciloc 150 Tablet,Ranitidine (150mg),Treatment of Gastroesophageal reflux disease (...,Headache Diarrhea Gastrointestinal disturbance,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Cadila Pharmaceuticals Ltd,34,37,29,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11820,11820,Zilarta-CT 40/6.25 Tablet,Azilsartan medoxomil (40mg) + Chlorthalidone (...,Hypertension (high blood pressure),Dizziness Tiredness Decreased potassium level ...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Micro Labs Ltd,67,33,0,...,0,0,0,0,0,0,0,0,0,0
11821,11821,Zipcoz Tablet,Chromium Picolinate (100mcg) + D-Chiro Inosito...,Polycystic ovarian syndrome (PCOS),Rash Nausea Paresthesia tingling or pricking s...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Lifeline Remedies India Pvt Ltd,0,80,20,...,0,0,0,0,0,0,0,0,0,0
11822,11822,Zestasil 100 Tablet,Sildenafil (100mg),Treatment of Erectile dysfunction,Flushing sense of warmth in the face ears neck...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",TBG pharma ltd,25,75,0,...,0,0,0,0,0,0,0,0,0,0
11823,11823,Zedruff Shampoo,Ketoconazole (2% w/v) + Zinc pyrithione (1% w/v),Treatment of Dandruff,Application site reactions burning irritation ...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Minova Life Sciences Pvt Ltd,0,0,100,...,0,0,0,0,0,0,0,0,0,0


In [17]:
dt[dt['Neurological_effects'] == 1].sum()

Unnamed: 0                   0.0
Medicine Name                0.0
Composition                  0.0
Uses                         0.0
Side_effects                 0.0
Image URL                    0.0
Manufacturer                 0.0
Excellent Review %           0.0
Average Review %             0.0
Poor Review %                0.0
Parsed_Composition           0.0
Standardized_Composition     0.0
Side_effects_List            0.0
Composition_String           0.0
Musculoskeletal_effects      0.0
Immune_system_effects        0.0
Gastrointestinal_effects     0.0
Neurological_effects         0.0
Respiratory_effects          0.0
Psychological_effects        0.0
Endocrine_effects            0.0
Dermatological_effects       0.0
Cardiovascular_effects       0.0
Renal_and_Urinary_effects    0.0
dtype: float64

In [19]:
final_data = pd.read_csv("FinalData.csv")
final_data.head()

,Unnamed: 0,Medicine Name,Composition,Uses,Side_effects,Image URL,Manufacturer,Excellent Review %,Average Review %,Poor Review %,Parsed_Composition,Standardized_Composition,Side_effects_List,Composition_String
0,0,Avastin 400mg Injection,Bevacizumab (400mg),Cancer of colon and rectum Non-small cell lun...,Rectal bleeding Taste change Headache Noseblee...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Roche Products India Pvt Ltd,22,56,22,{'Bevacizumab': '400mg'},{'Bevacizumab': '400mg'},['Rectal bleeding Taste change Headache Nosebl...,Bevacizumab:400mg
1,1,Augmentin 625 Duo Tablet,Amoxycillin (500mg) + Clavulanic Acid (125mg),Treatment of Bacterial infections,Vomiting Nausea Diarrhea Mucocutaneous candidi...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Glaxo SmithKline Pharmaceuticals Ltd,47,35,18,"{'Amoxycillin': '500mg', 'Clavulanic Acid': '1...","{'Amoxycillin': '500mg', 'Clavulanic Acid': '1...",['Vomiting Nausea Diarrhea Mucocutaneous candi...,Amoxycillin:500mg Clavulanic Acid:125mg
2,2,Azithral 500 Tablet,Azithromycin (500mg),Treatment of Bacterial infections,Nausea Abdominal pain Diarrhea,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Alembic Pharmaceuticals Ltd,39,40,21,{'Azithromycin': '500mg'},{'Azithromycin': '500mg'},['Nausea Abdominal pain Diarrhea'],Azithromycin:500mg
3,3,Ascoril LS Syrup,Ambroxol (30mg/5ml) + Levosalbutamol (1mg/5ml)...,Treatment of Cough with mucus,Nausea Vomiting Diarrhea Upset stomach Stomach...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Glenmark Pharmaceuticals Ltd,24,41,35,{},{},['Nausea Vomiting Diarrhea Upset stomach Stoma...,NaN
4,4,Aciloc 150 Tablet,Ranitidine (150mg),Treatment of Gastroesophageal reflux disease (...,Headache Diarrhea Gastrointestinal disturbance,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Cadila Pharmaceuticals Ltd,34,37,29,{'Ranitidine': '150mg'},{'Ranitidine': '150mg'},['Headache Diarrhea Gastrointestinal disturban...,Ranitidine:150mg


In [33]:
final_data.Side_effects_List.head()

0    ['Rectal bleeding Taste change Headache Nosebl...
1    ['Vomiting Nausea Diarrhea Mucocutaneous candi...
2                   ['Nausea Abdominal pain Diarrhea']
3    ['Nausea Vomiting Diarrhea Upset stomach Stoma...
4    ['Headache Diarrhea Gastrointestinal disturban...
Name: Side_effects_List, dtype: object

In [20]:
for domain in combined_dictionary.keys():
    final_data[domain] = 0

# Iterate through each row and check for side effects
for index, row in final_data.iterrows():
    # Normalize the side effect list to lowercase
    side_effect_list = ''.join(row['Side_effects_List']).split(' ')
    
    # Check each domain
    for domain, effects in combined_dictionary.items():
        # Check if any side effect in the row matches the domain's side effects
        if any(effect in effects for effect in side_effect_list):
                final_data.at[index, domain] = 1

In [21]:
final_data

,Unnamed: 0,Medicine Name,Composition,Uses,Side_effects,Image URL,Manufacturer,Excellent Review %,Average Review %,Poor Review %,...,Musculoskeletal_effects,Immune_system_effects,Gastrointestinal_effects,Neurological_effects,Respiratory_effects,Psychological_effects,Endocrine_effects,Dermatological_effects,Cardiovascular_effects,Renal_and_Urinary_effects
0,0,Avastin 400mg Injection,Bevacizumab (400mg),Cancer of colon and rectum Non-small cell lun...,Rectal bleeding Taste change Headache Noseblee...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Roche Products India Pvt Ltd,22,56,22,...,1,0,0,1,0,0,0,0,0,1
1,1,Augmentin 625 Duo Tablet,Amoxycillin (500mg) + Clavulanic Acid (125mg),Treatment of Bacterial infections,Vomiting Nausea Diarrhea Mucocutaneous candidi...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Glaxo SmithKline Pharmaceuticals Ltd,47,35,18,...,0,0,1,0,0,0,0,0,0,0
2,2,Azithral 500 Tablet,Azithromycin (500mg),Treatment of Bacterial infections,Nausea Abdominal pain Diarrhea,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Alembic Pharmaceuticals Ltd,39,40,21,...,0,0,1,0,0,0,0,0,0,0
3,3,Ascoril LS Syrup,Ambroxol (30mg/5ml) + Levosalbutamol (1mg/5ml)...,Treatment of Cough with mucus,Nausea Vomiting Diarrhea Upset stomach Stomach...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Glenmark Pharmaceuticals Ltd,24,41,35,...,1,0,1,1,0,1,0,1,1,0
4,4,Aciloc 150 Tablet,Ranitidine (150mg),Treatment of Gastroesophageal reflux disease (...,Headache Diarrhea Gastrointestinal disturbance,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Cadila Pharmaceuticals Ltd,34,37,29,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11820,11820,Zilarta-CT 40/6.25 Tablet,Azilsartan medoxomil (40mg) + Chlorthalidone (...,Hypertension (high blood pressure),Dizziness Tiredness Decreased potassium level ...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Micro Labs Ltd,67,33,0,...,0,0,0,0,0,1,0,0,0,0
11821,11821,Zipcoz Tablet,Chromium Picolinate (100mcg) + D-Chiro Inosito...,Polycystic ovarian syndrome (PCOS),Rash Nausea Paresthesia tingling or pricking s...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Lifeline Remedies India Pvt Ltd,0,80,20,...,0,0,1,1,0,1,0,0,1,0
11822,11822,Zestasil 100 Tablet,Sildenafil (100mg),Treatment of Erectile dysfunction,Flushing sense of warmth in the face ears neck...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",TBG pharma ltd,25,75,0,...,1,0,1,1,0,1,0,1,0,0
11823,11823,Zedruff Shampoo,Ketoconazole (2% w/v) + Zinc pyrithione (1% w/v),Treatment of Dandruff,Application site reactions burning irritation ...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Minova Life Sciences Pvt Ltd,0,0,100,...,0,0,0,0,0,0,0,0,0,0


In [22]:
final_data.sum()

C:\Users\Krish Tejani\AppData\Local\Temp\ipykernel_16480\94294402.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  final_data.sum()


Unnamed: 0                                                            69909400
Medicine Name                Avastin 400mg InjectionAugmentin 625 Duo Table...
Composition                  Bevacizumab (400mg)Amoxycillin  (500mg) +  Cla...
Uses                          Cancer of colon and rectum Non-small cell lun...
Side_effects                 Rectal bleeding Taste change Headache Noseblee...
Image URL                    https://onemg.gumlet.io/l_watermark_346,w_480,...
Manufacturer                 Roche Products India Pvt LtdGlaxo SmithKline P...
Excellent Review %                                                      455452
Average Review %                                                        422819
Poor Review %                                                           304229
Parsed_Composition           {'Bevacizumab': '400mg'}{'Amoxycillin': '500mg...
Standardized_Composition     {'Bevacizumab': '400mg'}{'Amoxycillin': '500mg...
Side_effects_List            ['Rectal bleeding Taste

In [1]:
final_data.to_csv("Final.csv")

NameError: name 'final_data' is not defined

In [38]:
final_data.columns

Index(['Unnamed: 0', 'Medicine Name', 'Composition', 'Uses', 'Side_effects',
       'Image URL', 'Manufacturer', 'Excellent Review %', 'Average Review %',
       'Poor Review %', 'Parsed_Composition', 'Standardized_Composition',
       'Side_effects_List', 'Composition_String', 'Musculoskeletal_effects',
       'Immune_system_effects', 'Gastrointestinal_effects',
       'Neurological_effects', 'Respiratory_effects', 'Psychological_effects',
       'Endocrine_effects', 'Dermatological_effects', 'Cardiovascular_effects',
       'Renal_and_Urinary_effects'],
      dtype='object')

In [3]:
model_data = pd.read_csv("Final.csv")
model_data.head()

,Unnamed: 0.1,Unnamed: 0,Medicine Name,Composition,Uses,Side_effects,Image URL,Manufacturer,Excellent Review %,Average Review %,...,Musculoskeletal_effects,Immune_system_effects,Gastrointestinal_effects,Neurological_effects,Respiratory_effects,Psychological_effects,Endocrine_effects,Dermatological_effects,Cardiovascular_effects,Renal_and_Urinary_effects
0,0,0,Avastin 400mg Injection,Bevacizumab (400mg),Cancer of colon and rectum Non-small cell lun...,Rectal bleeding Taste change Headache Noseblee...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Roche Products India Pvt Ltd,22,56,...,1,0,0,1,0,0,0,0,0,1
1,1,1,Augmentin 625 Duo Tablet,Amoxycillin (500mg) + Clavulanic Acid (125mg),Treatment of Bacterial infections,Vomiting Nausea Diarrhea Mucocutaneous candidi...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Glaxo SmithKline Pharmaceuticals Ltd,47,35,...,0,0,1,0,0,0,0,0,0,0
2,2,2,Azithral 500 Tablet,Azithromycin (500mg),Treatment of Bacterial infections,Nausea Abdominal pain Diarrhea,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Alembic Pharmaceuticals Ltd,39,40,...,0,0,1,0,0,0,0,0,0,0
3,3,3,Ascoril LS Syrup,Ambroxol (30mg/5ml) + Levosalbutamol (1mg/5ml)...,Treatment of Cough with mucus,Nausea Vomiting Diarrhea Upset stomach Stomach...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Glenmark Pharmaceuticals Ltd,24,41,...,1,0,1,1,0,1,0,1,1,0
4,4,4,Aciloc 150 Tablet,Ranitidine (150mg),Treatment of Gastroesophageal reflux disease (...,Headache Diarrhea Gastrointestinal disturbance,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Cadila Pharmaceuticals Ltd,34,37,...,0,0,1,0,0,0,0,0,0,0


In [4]:
# Impute missing values in the Composition_String column using available data from other composition-related columns
model_data['Composition_String'].fillna(model_data['Parsed_Composition'], inplace=True)
model_data['Composition_String'].fillna(model_data['Standardized_Composition'], inplace=True)
model_data['Composition_String'].fillna('Unknown', inplace=True)

# Confirm that there are no missing values in the Composition_String column
model_data['Composition_String'].isnull().sum()


0

In [7]:
X = model_data['Composition_String']
y = model_data[ 
    [
        'Neurological_effects',
        'Gastrointestinal_effects',
        'Cardiovascular_effects',
        'Dermatological_effects',
        'Psychological_effects',
        'Respiratory_effects',
        'Endocrine_effects',
        'Musculoskeletal_effects',
        'Renal_and_Urinary_effects',
        'Immune_system_effects',
    ]
]

tfidf = TfidfVectorizer(max_features=4000)  # Limit to 5000 features to reduce dimensionality
X_tfidf = tfidf.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.3, random_state=40)

clf = MultiOutputClassifier(RandomForestClassifier(random_state=35))
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

In [25]:
clf.score(X_test, y_test)

0.7373167981961668

In [10]:
y_pred

array([[1, 1, 1, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [11]:
print(X_test)

  (0, 18)	0.39589342953614126
  (0, 134)	0.34777944154152446
  (0, 620)	0.6113074819380122
  (0, 702)	0.5904413730572958
  (1, 108)	0.45165622594302723
  (1, 437)	0.8921920497106558
  (2, 83)	0.6344636904208382
  (2, 995)	0.7729526670744923
  (3, 56)	0.44341748307356244
  (3, 438)	0.8963151988584747
  (4, 24)	0.4105994638665541
  (4, 121)	0.3441315298375495
  (4, 134)	0.2676805984696988
  (4, 322)	0.49347792651571515
  (4, 686)	0.44395564939317783
  (4, 897)	0.4480084653895527
  (7, 105)	0.5588792849773713
  (7, 187)	0.8292490246139469
  (8, 13)	0.23871455765754948
  (8, 115)	0.4132736769903874
  (8, 121)	0.23561178537191155
  (8, 168)	0.36178663519928056
  (8, 275)	0.39404261149554853
  (8, 386)	0.3407599046651719
  (8, 865)	0.39404261149554853
  :	:
  (2351, 757)	0.6729324314362686
  (2352, 121)	0.3688120884731061
  (2352, 542)	0.9295039770738508
  (2353, 75)	0.49543447398687085
  (2353, 951)	0.8686453142597111
  (2354, 120)	0.3743484419765061
  (2354, 297)	0.9272881127188908
  (2355

In [13]:
# Sample Composition_String for prediction
sample_input = ["Epirubicin hydrochloride 50mg"]

# Transform the sample input using the fitted TF-IDF vectorizer
sample_tfidf = tfidf.transform(sample_input)

# Predict side effects for the sample input
sample_prediction = clf.predict(sample_tfidf)

# Convert the result to a readable format
predicted_side_effects = {label: bool(pred) for label, pred in zip(y.columns, sample_prediction[0])}

print("Predicted Side Effects:")
print(predicted_side_effects)

Predicted Side Effects:
{'Neurological_effects': True, 'Gastrointestinal_effects': True, 'Cardiovascular_effects': False, 'Dermatological_effects': False, 'Psychological_effects': False, 'Respiratory_effects': False, 'Endocrine_effects': False, 'Musculoskeletal_effects': False, 'Renal_and_Urinary_effects': False, 'Immune_system_effects': False}


In [9]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.decomposition import TruncatedSVD

# Assuming model_data, X, and y are already defined as in your code

# 1. Improved TF-IDF with n-grams
tfidf = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),  # Include bigrams
    min_df=2,            # Ignore terms that appear in less than 2 documents
    max_df=0.9,          # Ignore terms that appear in more than 90% of documents
    sublinear_tf=True    # Apply sublinear tf scaling (1+log(tf))
)

X_tfidf = tfidf.fit_transform(X)

# 2. Apply dimensionality reduction (optional)
svd = TruncatedSVD(n_components=300)
X_svd = svd.fit_transform(X_tfidf)

# 3. Split the data
X_train, X_test, y_train, y_test = train_test_split(X_svd, y, test_size=0.2, random_state=42)

# 4. Try multiple models and choose the best

# 4.1 Tuned Random Forest - FIXED PARAMETER SETTING
rf_clf = MultiOutputClassifier(RandomForestClassifier(
    n_estimators=200,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    bootstrap=True,
    class_weight='balanced',
    n_jobs=-1,
    random_state=42
))

# 4.2 Gradient Boosting
gb_clf = MultiOutputClassifier(GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    subsample=0.8,
    random_state=42
))

# 4.3 SVM
svm_clf = MultiOutputClassifier(LinearSVC(
    C=1.0,
    class_weight='balanced',
    dual=False,
    max_iter=10000,
    random_state=42
))

# 4.4 Neural Network
nn_clf = MultiOutputClassifier(MLPClassifier(
    hidden_layer_sizes=(100, 50),
    activation='relu',
    solver='adam',
    alpha=0.0001,
    batch_size='auto',
    learning_rate='adaptive',
    max_iter=1000,
    random_state=42
))

# 5. Train and evaluate models
models = {
    'Random Forest (Tuned)': rf_clf,
    'Gradient Boosting': gb_clf,
    'SVM': svm_clf,
    'Neural Network': nn_clf
}

best_accuracy = 0
best_model_name = None
best_model = None

for name, model in models.items():
    # Try to fit the model, catch any errors
    try:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test.values.flatten(), y_pred.flatten())
        
        print(f"{name} Accuracy: {accuracy:.4f}")
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model_name = name
            best_model = model
    except Exception as e:
        print(f"Error fitting {name}: {str(e)}")

if best_model is not None:
    print(f"\nBest Model: {best_model_name} with accuracy: {best_accuracy:.4f}")

    # 6. Fine-tune the best model (if it's Random Forest)
    if best_model_name == 'Random Forest (Tuned)':
        param_grid = {
            'estimator__n_estimators': [200, 300],
            'estimator__max_depth': [15, 20, 25],
            'estimator__min_samples_split': [2, 5, 10],
            'estimator__min_samples_leaf': [1, 2, 4]
        }
        
        grid_search = GridSearchCV(
            MultiOutputClassifier(RandomForestClassifier(random_state=42)), 
            param_grid=param_grid,
            cv=3,
            scoring='accuracy',
            n_jobs=-1
        )
        
        grid_search.fit(X_train, y_train)
        
        print("\nBest parameters from GridSearchCV:")
        print(grid_search.best_params_)
        
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)
        final_accuracy = accuracy_score(y_test.values.flatten(), y_pred.flatten())
        print(f"Final tuned accuracy: {final_accuracy:.4f}")

    # 7. Get detailed metrics for the best model
    y_pred = best_model.predict(X_test)
    print("\nClassification Report:")
    for i, col in enumerate(y.columns):
        print(f"\n{col}:")
        print(classification_report(y_test.iloc[:, i], y_pred[:, i]))

    # 8. Feature importance analysis (for Random Forest)
    if best_model_name == 'Random Forest (Tuned)':
        print("\nTop 20 Important Features:")
        for i, estimator in enumerate(best_model.estimators_):
            if i == 0:  # Just show for the first target for brevity
                importances = estimator.feature_importances_
                
                if hasattr(tfidf, 'get_feature_names_out'):
                    feature_names = tfidf.get_feature_names_out()
                else:
                    feature_names = tfidf.get_feature_names()
                
                # Map back through SVD if used
                if 'svd' in locals():
                    # This is simplified - true feature importance with SVD is more complex
                    print("Note: Feature importance with dimensionality reduction is approximate")
                
                indices = np.argsort(importances)[::-1][:20]
                for idx in indices:
                    print(f"{feature_names[idx]} = {importances[idx]:.4f}")
                break
else:
    print("No model could be fitted successfully.")

Random Forest (Tuned) Accuracy: 0.9054
Gradient Boosting Accuracy: 0.9350
SVM Accuracy: 0.8629
Neural Network Accuracy: 0.9380

Best Model: Neural Network with accuracy: 0.9380

Classification Report:

Neurological_effects:
              precision    recall  f1-score   support

           0       0.83      0.96      0.89      1146
           1       0.96      0.81      0.88      1219

    accuracy                           0.88      2365
   macro avg       0.89      0.89      0.88      2365
weighted avg       0.89      0.88      0.88      2365


Gastrointestinal_effects:
              precision    recall  f1-score   support

           0       0.79      0.94      0.86       882
           1       0.96      0.85      0.90      1483

    accuracy                           0.88      2365
   macro avg       0.87      0.89      0.88      2365
weighted avg       0.90      0.88      0.89      2365


Cardiovascular_effects:
              precision    recall  f1-score   support

           0   

In [10]:
# Print final accuracy and model details
print("\n========== FINAL MODEL RESULTS ==========")
print(f"Selected Model: {best_model_name}")
print(f"Model Accuracy: {best_accuracy:.4f} ({best_accuracy*100:.2f}%)")

# Get classification metrics for each target
y_pred = best_model.predict(X_test)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

metrics = {}
for i, col in enumerate(y.columns):
    metrics[col] = {
        'accuracy': accuracy_score(y_test.iloc[:, i], y_pred[:, i]),
        'precision': precision_score(y_test.iloc[:, i], y_pred[:, i], zero_division=0),
        'recall': recall_score(y_test.iloc[:, i], y_pred[:, i], zero_division=0),
        'f1': f1_score(y_test.iloc[:, i], y_pred[:, i], zero_division=0)
    }

# Print metrics per target
print("\nPerformance by Target:")
for target, metric in metrics.items():
    print(f"{target}:")
    print(f"  Accuracy:  {metric['accuracy']:.4f}")
    print(f"  Precision: {metric['precision']:.4f}")
    print(f"  Recall:    {metric['recall']:.4f}")
    print(f"  F1 Score:  {metric['f1']:.4f}")

# Print model parameters (if it's a random forest)
if best_model_name == 'Random Forest (Tuned)':
    print("\nModel Parameters:")
    # Get parameters from the first estimator (they should all be the same)
    params = best_model.estimators_[0].get_params()
    important_params = ['n_estimators', 'max_depth', 'min_samples_split', 
                        'min_samples_leaf', 'max_features']
    for param in important_params:
        print(f"  {param}: {params[param]}")


========== FINAL MODEL RESULTS ==========
Selected Model: Neural Network
Model Accuracy: 0.9380 (93.80%)

Performance by Target:
Neurological_effects:
  Accuracy:  0.8850
  Precision: 0.9566
  Recall:    0.8138
  F1 Score:  0.8794
Gastrointestinal_effects:
  Accuracy:  0.8837
  Precision: 0.9576
  Recall:    0.8523
  F1 Score:  0.9019
Cardiovascular_effects:
  Accuracy:  0.9366
  Precision: 0.9545
  Recall:    0.7831
  F1 Score:  0.8603
Dermatological_effects:
  Accuracy:  0.8727
  Precision: 0.9170
  Recall:    0.4655
  F1 Score:  0.6175
Psychological_effects:
  Accuracy:  0.9125
  Precision: 0.9632
  Recall:    0.7746
  F1 Score:  0.8587
Respiratory_effects:
  Accuracy:  0.9784
  Precision: 0.9699
  Recall:    0.7330
  F1 Score:  0.8350
Endocrine_effects:
  Accuracy:  0.9852
  Precision: 0.9697
  Recall:    0.6598
  F1 Score:  0.7853
Musculoskeletal_effects:
  Accuracy:  0.9531
  Precision: 0.9601
  Recall:    0.8021
  F1 Score:  0.8740
Renal_and_Urinary_effects:
  Accuracy:  0.9848

In [11]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import accuracy_score, classification_report

# Assuming model_data, X, and y are already defined as in your code

# 1. Improved TF-IDF with n-grams
tfidf = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),  # Include bigrams
    min_df=2,            # Ignore terms that appear in less than 2 documents
    max_df=0.9,          # Ignore terms that appear in more than 90% of documents
    sublinear_tf=True    # Apply sublinear tf scaling (1+log(tf))
)

X_tfidf = tfidf.fit_transform(X)

# 2. Apply dimensionality reduction
svd = TruncatedSVD(n_components=300)
X_svd = svd.fit_transform(X_tfidf)

# 3. Split the data
X_train, X_test, y_train, y_test = train_test_split(X_svd, y, test_size=0.2, random_state=42)

# 4. Create optimized Random Forest model
rf_model = MultiOutputClassifier(RandomForestClassifier(
    n_estimators=200,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    bootstrap=True,
    class_weight='balanced',
    n_jobs=-1,
    random_state=42
))

# 5. Train the model
rf_model.fit(X_train, y_train)

# 6. Evaluate the model
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test.values.flatten(), y_pred.flatten())
print(f"Model Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")



Model Accuracy: 0.9056 (90.56%)


In [15]:
def predict_health_effects(composition_text):
    # Preprocess the input text the same way training data was processed
    X_input = tfidf.transform([composition_text])
    X_input_svd = svd.transform(X_input)
    
    # Make predictions
    y_pred = rf_model.predict(X_input_svd)
    
    # Convert predictions to a more user-friendly format
    results = {}
    for i, col in enumerate(y.columns):
        # Convert from 0/1 to False/True
        results[col] = bool(y_pred[0][i])
    
    return results

composition_text = "Epirubicin hydrochloride 50mg"
print(predict_health_effects(composition_text))

{'Neurological_effects': False, 'Gastrointestinal_effects': True, 'Cardiovascular_effects': False, 'Dermatological_effects': False, 'Psychological_effects': True, 'Respiratory_effects': False, 'Endocrine_effects': False, 'Musculoskeletal_effects': False, 'Renal_and_Urinary_effects': False, 'Immune_system_effects': False}
